In [4]:
# Parameters
input_data_path = ""
batch = ""
label = ""

Jupyter notebook for conos integration algoritm

Author: Erno Hänninen

Title: run_fastmnn.ipynb

Created: 2022-11-30

In [5]:
require(batchelor)
require(Seurat)

In [ ]:
sobj = readRDS(input_data_path)
sobj

In [7]:
# This function is from https://github.com/theislab/scib-pipeline/blob/0c7be53b1000864fcd31a7b7594f9a5071204233/scripts/integration/integration.R
runFastMNN = function(sobj, batch) {
  suppressPackageStartupMessages({
    require(batchelor)
  })

  if (is.null(sobj@assays$RNA)) {
    # Seurat v4
    expr <- GetAssayData(sobj, slot = "data")
  } else {
    # Seurat v3
    expr <- sobj@assays$RNA@data
  }

  sce <- fastMNN(expr, batch = sobj@meta.data[[batch]])
  corrected_data <- assay(sce, "reconstructed")

  if (is.null(sobj@assays$RNA)) {
    # Seurat v4
    sobj <- SetAssayData(sobj, slot = "data", new.data = as.matrix(corrected_data))
    sobj@reductions['X_emb'] <- CreateDimReducObject(reducedDim(sce, "corrected"), key = 'fastmnn_')
  } else {
    # Seurat v3
    sobj@assays$RNA <- CreateAssayObject(corrected_data)
    sobj[['X_emb']] <- CreateDimReducObject(reducedDim(sce, "corrected"), key = 'fastmnn_')
  }

  return(sobj)
}


In [ ]:

if(file.exists(hvg)) {
	print("EXISTS")
	hvg<-unlist(readRDS(hvg), use.names=FALSE)
	print(length(hvg))
	if(length(hvg) > 0){
		sobj <- subset(sobj, features=hvg)
	}
} 

fastmnn_sobj=runFastMNN(sobj, batch)
saveRDS(fastmnn_sobj, file="../Integrated_adata/fastmnn_output.rds")